In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 66.7 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def teleport(phi, theta):
    """Quantum teleportation circuit.

    Args:
        phi (float): Rotation angle around the Z-axis for the initial state.
        theta (float): Rotation angle around the Y-axis for the initial state.

    Returns:
        array[float]: Probabilities of the final state on Bob's qubit.
    """
    # Step 1: Prepare Alice's unknown qubit on wire 0
    qml.RY(theta, wires=0)
    qml.RZ(phi, wires=0)

    # Step 2: Create Bell pair between qubits 1 (Alice) and 2 (Bob)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])

    # Step 3: Bell measurement on qubits 0 and 1 (Alice's qubits)
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

    # Step 4: Measure and apply conditional corrections on qubit 2 (Bob)
    m0 = qml.measure(0)
    m1 = qml.measure(1)
    qml.cond(m1, qml.PauliX)(wires=2)
    qml.cond(m0, qml.PauliZ)(wires=2)

    # Step 5: Return the probabilities of the final state on Bob's qubit
    return qml.probs(wires=2)  # Changed from qml.state() to qml.probs(wires=2)

# Test angles (can be varied)
phi = np.pi / 4
theta = np.pi / 3

teleported_state = teleport(phi, theta)

# Also compute the original state for comparison
def get_original_state(phi, theta):
    state_dev = qml.device("default.qubit", wires=1)
    @qml.qnode(state_dev)
    def prepare():
        qml.RY(theta, wires=0)
        qml.RZ(phi, wires=0)
        return qml.state()
    return prepare()

original_state = get_original_state(phi, theta)

print("Original state on qubit 0:\n", original_state)
print("\n Teleported state probabilities on qubit 2:\n", teleported_state) # Changed output label to reflect the change

Original state on qubit 0:
 [0.80010315-0.33141357j 0.46193977+0.19134172j]

 Teleported state probabilities on qubit 2:
 [0.75 0.25]
